### 💻 **<i>Notebook 04:</i> Gradio and Interface Deployment**

**For running on Google Colab only, run the code cell below:**

In [ ]:
import os
from dotenv import load_dotenv
try:
    # This import ONLY works on the Colab web interface
    from google.colab import userdata

    print("Loading API keys from Colab Secrets...")

    # --- Key Retrieval and Injection into os.environ ---
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_CSE_ID'] = userdata.get('GOOGLE_CSE_ID')
    os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
    os.environ['LANGCHAIN_PROJECT'] = "retrogaming-qa-bot"

    # --- Verification ---
    if not os.environ.get('OPENAI_API_KEY'):
        raise ValueError("OPENAI_API_KEY was not found in Colab Secrets.")

    print("✅ All API keys were successfully loaded from Colab Secrets.")

except ImportError:
    # Fallback for local environments (local VS Code)
    print("Loading API keys from local .env file (VS Code/Local Mode).")
    load_dotenv()

# --- Common Configurations ---
CHROMA_PATH = "./chroma_db_retrogaming"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"

# Here, the rest of the notebook variables are defined (llm, embeddings, etc.)
# ... (should continue defining LLM/RAG configuration variables)

A carregar chaves API do Colab Secrets...
✅ Todas as chaves API foram carregadas com sucesso a partir do Colab Secrets.


##### **00 - Imports and Setup**

In [2]:
pip install langchain_openai langchain_community langchain_classic gradio python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import os
from dotenv import load_dotenv
import gradio as gr

# LangChain Imports for Agent Reconstruction
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.tools import Tool
from langchain_classic.agents import initialize_agent, AgentType
from langchain_classic.memory import ConversationBufferWindowMemory

# Imports for Hybrid Agent Tools
from langchain_community.tools import GoogleSearchRun
from langchain_community.utilities import GoogleSearchAPIWrapper


# ====================================================================
# 01 - Imports, Memory, and Agent Reconstruction
# ====================================================================

# Load environment variables (critical for API keys)
load_dotenv()

# --- Memory Definition ---
# k=3 for conciseness and token management
conversational_memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    return_messages=True
)
print("✅ Configuration and Memory loaded.")

✅ Configuration and Memory loaded.


/tmp/ipython-input-3250473048.py:26: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferWindowMemory(


In [5]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.4 MB/

##### **01 - Agent Reconstruction**

In [3]:
# --- Agent Reconstruction ---
# This block is essential because the Gradio application runs the entire notebook,
# and the agent_executor from Notebook 03 does not persist across sessions.

# 1. Configuration
CHROMA_PATH = "./chroma_db_retrogaming"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"

llm = ChatOpenAI(model=LLM_MODEL, temperature=0.0)

# 2. RAG Tool Setup (Reconstruction)
print("\nReconstructing Retriever and Tools...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vector_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# RAG Wrapper function
def get_retrogaming_docs(query: str) -> str:
    """Fetches documents from the RAG knowledge base."""
    docs = retriever.invoke(query)
    formatted_docs = []
    for doc in docs:
        source_type = doc.metadata.get('source_type', 'unknown')
        formatted_docs.append(f"[{source_type.upper()}] {doc.page_content}")
    return "\n\n".join(formatted_docs)

# RAG Tool Definition
retrogaming_qa_tool = Tool(
    name="retrogaming_knowledge_base",
    func=get_retrogaming_docs,
    description="Tool to search and retrieve specific information from the internal knowledge base (YouTube transcripts or Web tutorials) about emulators, settings, and consoles. Use this FIRST."
)

# 3. Google Search Tool Setup (Reconstruction)
search_wrapper = GoogleSearchAPIWrapper()
google_search_tool = GoogleSearchRun(
    name="google_search",
    api_wrapper=search_wrapper,
    description="Use this tool to search the internet for current or highly specific information when the internal knowledge base does not have the answer. Useful for new game specs or up-to-date emulator news."
)

# 4. Final List of Tools
tools = [retrogaming_qa_tool, google_search_tool]

# 5. System Prompt (Copied from Notebook 03)
SYSTEM_TEMPLATE = (
    "You are a comprehensive expert retrogaming assistant and tuning specialist. Your expertise covers "
    "tuning old video games from various platforms (including PC) for better resolution and modern "
    "compatibility on PC, Mac, or the original native console hardware. "

    "***INITIAL CONVERSATION GUIDANCE:***"

    "1. **PC/Mac Path Flow:** If the user chooses PC or Mac:"
    "   - **Gather Specs:** IMMEDIATELY ask for the user's current **computer specs** (CPU/GPU/RAM) and their **monitor's native resolution**. **CRITICAL RULE:** If the user states they do not know how to find their computer specs, you MUST provide clear, step-by-step answers on how to find this information on their operating system (Windows or macOS)."
    "   - **Advice:** Once specs are known, use your tools to provide information on the available **emulators** (like PCSX2, Dolphin, etc.) and **tuning specs** tailored to their hardware for the specific game (e.g., Jak 2 Renegade)."

    "2. **Native Console Path Flow:** If the user chooses the native console:"
    "   - **Advice:** Use your tools to reply with the best options to output that system to its best possible resolution, naming **external third-party devices** like HDMI adapters or image/signal converters."

    "***HYBRID CONSTRAINT (RAG & SEARCH PRIORITY):***"
    "3. **PRIORITY:** You **MUST** attempt to use the tool **'retrogaming_knowledge_base'** first."
    "4. **FALLBACK:** If the required information is **NOT** found in the RAG context, you **MUST** use the tool **'google_search'** to find a reliable external source before formulating the final answer."
    "5. **TRANSPARENCY:** If the final answer is based on the Google search, briefly mention that the information was retrieved from an updated search."

    "\n\nChat History: {chat_history}"
    "\n\nUser Question: {input}"
)

# 6. Define the Agent Executor
agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=conversational_memory,
    agent_kwargs={"system_message": SYSTEM_TEMPLATE}
)

print("✅ Hybrid Agent Executor reconstructed and ready for Gradio.")


Reconstructing Retriever and Tools...


/tmp/ipython-input-2637959365.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)


✅ Hybrid Agent Executor reconstructed and ready for Gradio.


/tmp/ipython-input-2637959365.py:36: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the `langchain-google-community package and should be used instead. To use it run `pip install -U `langchain-google-community` and import as `from `langchain_google_community import GoogleSearchAPIWrapper``.
  search_wrapper = GoogleSearchAPIWrapper()
/tmp/ipython-input-2637959365.py:37: LangChainDeprecationWarning: The class `GoogleSearchRun` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the `langchain-google-community package and should be used instead. To use it run `pip install -U `langchain-google-community` and import as `from `langchain_google_community import GoogleSearchRun``.
  google_search_tool = GoogleSearchRun(
/tmp/ipython-input-2637959365.py:71: LangChainDeprecationWarning: LangChain agents will continue to be

##### **02 - Conversational Flow Logic for Gradio**

In [4]:
# ====================================================================
# 02 - Conversational Flow Logic for Gradio
# ====================================================================

# The mandatory introductory question
MANDATORY_QUESTION = (
    "Hello! I am your expert retrogaming assistant. I can help you tune old games "
    "for modern setups. To start, which platform do you wish to play the game on? "
    "A **PC/Mac using an emulator** or the **native console hardware**?"
)

# This function is called every time the user sends a message in the Gradio interface
def chatbot_response(message, chat_history):
    # CRITICAL FLOW CONTROL: Bypass the agent on Turn 1 to ask the mandatory question
    if not chat_history:
        # Clear the agent's memory for a fresh start
        conversational_memory.clear()

        # Output the mandatory question directly (enforces the flow)
        return MANDATORY_QUESTION

    # Use the full Hybrid Agent Executor for all subsequent turns (Turn 2 onwards)
    try:
        # The agent uses its tools (RAG/Google Search) and memory to generate a response
        response = agent_executor.invoke({"input": message})

        return response['output']

    except Exception as e:
        # Catch errors gracefully (especially API or Context Length errors)
        print(f"Agent execution error: {e}")
        return "I apologize, but I encountered an internal error while processing your request. Please try again."



##### **03 - Interface Deployment**

In [5]:
# ====================================================================
# 03 - Gradio Interface Deployment
# ====================================================================

# Create the Gradio Interface
gr.ChatInterface(
    fn=chatbot_response,
    title="🕹️ RetroTune AI Assistant (Hybrid Chatbot)",
    description="Your expert guide to tuning retro games for modern displays, using RAG and Google Search.",

    # Examples help the user understand the expected input format
    examples=[
        "I want to play Silent Hill 2.",
        "I choose PC/Mac.",
        "My specs are i7 12th gen, 32GB RAM, and my monitor is 4K. I will use a controller.",
        "I want to play on my original PlayStation2 console."
    ],
).launch(share=True) # share=True generates a public link for presentation

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0a0b3d6627bacb9930.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
